In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
df=sns.load_dataset("diamonds")

In [3]:
df

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [4]:
df=pd.get_dummies(df, drop_first=True)

In [5]:
df

,carat,depth,table,price,x,y,z,cut_Premium,cut_Very Good,cut_Good,...,color_H,color_I,color_J,clarity_VVS1,clarity_VVS2,clarity_VS1,clarity_VS2,clarity_SI1,clarity_SI2,clarity_I1
0,0.23,61.5,55.0,326,3.95,3.98,2.43,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0.21,59.8,61.0,326,3.89,3.84,2.31,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0.23,56.9,65.0,327,4.05,4.07,2.31,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,0.29,62.4,58.0,334,4.20,4.23,2.63,1,0,0,...,0,1,0,0,0,0,1,0,0,0
4,0.31,63.3,58.0,335,4.34,4.35,2.75,0,0,1,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53935,0.72,60.8,57.0,2757,5.75,5.76,3.50,0,0,0,...,0,0,0,0,0,0,0,1,0,0
53936,0.72,63.1,55.0,2757,5.69,5.75,3.61,0,0,1,...,0,0,0,0,0,0,0,1,0,0
53937,0.70,62.8,60.0,2757,5.66,5.68,3.56,0,1,0,...,0,0,0,0,0,0,0,1,0,0
53938,0.86,61.0,58.0,2757,6.15,6.12,3.74,1,0,0,...,1,0,0,0,0,0,0,0,1,0


In [6]:
X=df.drop("price",1)
y=df["price"]

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import mean_squared_error
from pyod.models.knn import KNN

In [16]:
contamination=[0.01, 0.05, 0.1]
n_neighbors=[20]
method=["largest","mean","median"]
distance=[1,2,float('inf')]

In [12]:
def knn_detect_rmse(X, y, contamination, n_neighbors, method, distance, threshold=0.55):
    
    X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=10)
    
    qt=QuantileTransformer(output_distribution="normal")
    X_train_qt=qt.fit_transform(X_train)
    X_test_qt=qt.transform(X_test)
    
    X_train.loc[:,:]=X_train
    X_test.loc[:,:]=X_test
    
    
    variations=[ [a,b,c,d] for a in contamination for b in n_neighbors for c in distance for d in method]
    
    if threshold==None:
        
        for j,k,l,i in variations:
            knn=KNN(contamination=j, n_neighbors=k, method=i ,p=l, n_jobs=-1)
            knn.fit(X_train)
            X_train=X_train[knn.labels_==0]
            y_train=y_train.loc[X_train.index.to_list()]
            
            lr=LinearRegression()
            lr.fit(X_train, y_train)
            pred=lr.predict(X_test)
            rmse=np.sqrt(mean_squared_error(y_test, pred))
        
            print(f"{j,k,l,i} : {round(rmse,4)}")
        
    else:
        
        for j,k,l,i in variations:
            knn=KNN(contamination=j, n_neighbors=k, method=i ,p=l, n_jobs=-1)
            knn.fit(X_train)
            probs=knn.predict_proba(X_train)
            X_train=X_train[probs[:,1]<threshold]
            y_train=y_train.loc[X_train.index.to_list()]
            
            lr=LinearRegression()
            lr.fit(X_train, y_train)
            pred=lr.predict(X_test)
            rmse=np.sqrt(mean_squared_error(y_test, pred))
        
            print(f"{j,k,l,i} : {round(rmse,4)}")
    

In [17]:
knn_detect_rmse(X, y, contamination, n_neighbors, method, distance, threshold=None)

(0.01, 20, 1, 'largest') : 1385.9362
(0.01, 20, 1, 'mean') : 1776.9973
(0.01, 20, 1, 'median') : 1948.682
(0.01, 20, 2, 'largest') : 1935.655
(0.01, 20, 2, 'mean') : 1966.4638
(0.01, 20, 2, 'median') : 2038.3279
(0.01, 20, inf, 'largest') : 2039.0706
(0.01, 20, inf, 'mean') : 2039.0706
(0.01, 20, inf, 'median') : 2039.0706
(0.05, 20, 1, 'largest') : 2016.7643
(0.05, 20, 1, 'mean') : 1994.1204
(0.05, 20, 1, 'median') : 1888.8434
(0.05, 20, 2, 'largest') : 2311.6248
(0.05, 20, 2, 'mean') : 2316.2885
(0.05, 20, 2, 'median') : 2257.4653
(0.05, 20, inf, 'largest') : 2257.4653
(0.05, 20, inf, 'mean') : 2209.3768
(0.05, 20, inf, 'median') : 2209.3768
(0.1, 20, 1, 'largest') : 2297.3608
(0.1, 20, 1, 'mean') : 2478.3005
(0.1, 20, 1, 'median') : 2537.1213
(0.1, 20, 2, 'largest') : 2402.7517
(0.1, 20, 2, 'mean') : 2318.9432
(0.1, 20, 2, 'median') : 2148.9184
(0.1, 20, inf, 'largest') : 2148.9184
(0.1, 20, inf, 'mean') : 4550.5373
(0.1, 20, inf, 'median') : 4292.788


In [ ]:
# (0.01, 20, 1, 'largest') : 1385.9362

In [18]:
knn=KNN(contamination=0.01, n_neighbors=20, method="largest", p=1, n_jobs=-1)
knn.fit(df)
probs=knn.predict_proba(df)
df[probs[:,1]>0.30].index

Int64Index([    0,     1,     2, 23980, 23981, 24067, 24932, 24954, 24955,
            24956,
            ...
            27742, 27743, 27744, 27745, 27746, 27747, 27748, 27749, 48410,
            49189],
           dtype='int64', length=748)

In [19]:
df[knn.labels_==1]

,carat,depth,table,price,x,y,z,cut_Premium,cut_Very Good,cut_Good,...,color_H,color_I,color_J,clarity_VVS1,clarity_VVS2,clarity_VS1,clarity_VS2,clarity_SI1,clarity_SI2,clarity_I1
1,0.21,59.8,61.0,326,3.89,3.84,2.31,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0.23,56.9,65.0,327,4.05,4.07,2.31,0,0,1,...,0,0,0,0,0,1,0,0,0,0
15951,1.14,57.5,67.0,6381,0.00,0.00,0.00,0,0,0,...,0,0,0,0,0,1,0,0,0,0
22831,3.00,57.0,64.0,10863,9.38,9.31,5.33,0,0,1,...,0,1,0,0,0,0,0,0,0,1
23292,2.00,58.5,68.0,11322,8.26,8.15,4.80,0,0,0,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27749,2.29,60.8,60.0,18823,8.50,8.47,5.16,1,0,0,...,0,1,0,0,0,0,1,0,0,0
48410,0.51,61.8,54.7,1970,5.12,5.15,31.80,0,1,0,...,0,0,0,0,0,1,0,0,0,0
49189,0.51,61.8,55.0,2075,5.15,31.80,5.12,0,0,0,...,0,0,0,0,0,1,0,0,0,0
52860,0.50,79.0,73.0,2579,5.21,5.18,4.09,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [20]:
mask=df[knn.labels_==1].index.to_list()

In [21]:
pd.DataFrame({"index":mask, "prob":probs[:,1][mask]})

,index,prob
0,1,0.331815
1,2,0.338761
2,15951,0.205699
3,22831,0.198531
4,23292,0.255564
...,...,...
535,27749,0.829638
536,48410,0.368756
537,49189,0.340869
538,52860,0.292175
